<a href="https://colab.research.google.com/github/carlosfmorenog/CMM202/blob/master/CMM202_CW_P1_2022/03_CMM202_Coursework_P1_Model_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Carlos

Student ID: N/A

# Coursework Part 1 Template

The aim of this coursework is to design a program that simulates how a vending machine works. To do so, you will be provided with two small data files in .csv format, which you have to manipulate in order to get a data frame which will be used as the input of your vending machine simulator program.

You can copy this template to your Colab, Kaggle and/or Github account and use it as a starting point for your work.

Once you are done, you can either share the link to your notebook (paste it on a .txt file and submit it) or you can download the .ipynb file and submit it instead.

## Section 1: Data loading and manipulation

You will work with two files that are located in my dropbox:
1. `vending_machine.csv`: A table that shows the code to be entered in the vending machine keyboard to get a certain snack. It also contains the amount of products in each slot. The file can be read from this addres: https://www.dropbox.com/s/vy61ldio3t02lqy/vending_machine.csv?raw=1

2. `stock.csv`: A table with the name of each snack (without capitalisation), the category and the price. The file can be read from this addres: https://www.dropbox.com/s/qwl4i3slj4mklsj/stock.csv?raw=1

By running the following code cells, you can to read and print the two files as pandas data frames:

In [ ]:
import pandas as pd

In [ ]:
vending_machine = pd.read_csv('https://www.dropbox.com/s/vy61ldio3t02lqy/vending_machine.csv?raw=1')
vending_machine

,machine_code,snack,amount
0,A1,NaN,NaN
1,A2,Diet Coke,7.0
2,A3,Coke,2.0
3,A4,Irn-bru,5.0
4,B1,Mars,5.0
5,B2,Mars,7.0
6,B3,Milky Way,8.0
7,B4,NaN,NaN
8,C1,Sea Salt Crisps,2.0
9,C2,NaN,NaN


In [ ]:
stock = pd.read_csv('https://www.dropbox.com/s/qwl4i3slj4mklsj/stock.csv?raw=1')
stock

,snack,category,price
0,coca cola's coke,drinks,3.70
1,coca-cola's diet coke,drinks,3.75
2,a.g. bar's irn-bru,drinks,3.00
3,Forrest Mars' mars,chocolate,2.90
4,mars inc's milky way,chocolate,2.85
5,mackies' sea salt crisps,crisps,2.15
6,walkers' shortbread,biscuits,2.50
7,pepsico's doritos,crisps,2.50


### Question 1: Standardising the column names

The snack names are not written exactly in the same on both data frames! Most notably, you can see that in the `stock` data frame the snack has the brand before the snack name, whereas in the `vending_machine` data frame the snacks are capitalised and do not have the brand name before. Therefore, you need to create and **apply** a function which cleans the `snack` entries of the `stock` data frame to make them look **the same way** as in the `vending_machine` data frame!

In [ ]:
## use this cell to standardise the product names in the "stock" data frame
# My solution, clearly not the best!
def fix_name(x):
  if "'" in x:
      y = x.split("'")[1] #take out the brand name
      if y[0]=='s':# if the first letter after the split is an s, then the s is after the apostrophe
        z = y.split("s ")[1]
      else: # get everything after the space (words may have more spaces such as sea salt crisps)
        prov = y.split(" ")[1:]
        z = ""
        for i, word in enumerate(prov):
            if i+1==len(prov):
                z=z+word
            else:
                z=z+word+" "
      if '-' not in z:
        return z.title()
      else:
        return z.capitalize()
  else:
    return x

stock['snack'] = stock['snack'].apply(fix_name)
stock

,snack,category,price
0,Coke,drinks,3.70
1,Diet Coke,drinks,3.75
2,Irn-bru,drinks,3.00
3,Mars,chocolate,2.90
4,Milky Way,chocolate,2.85
5,Sea Salt Crisps,crisps,2.15
6,Shortbread,biscuits,2.50
7,Doritos,crisps,2.50


In [ ]:
# Another option, way simpler than the previous one!
# This one simply checks in both lists and substitutes
def change_name(x):
    lst = ''
    for s in stock[x]:
        for v in vending_machine[x].astype('str'):
            if (v.lower() in s.lower()) and (v not in lst):
                lst = v
    return lst
stock['snack'] = stock['snack'].apply(change_name)
stock

In [ ]:
# Another solution, using the .find methos
def change_name(x):
    import string
    snack_name_pos = x.find("'")
    snack_name = string.capwords(x[snack_name_pos+2:].strip())
    return snack_name
stock_df = stock[['snack','category','price']]
stock_df['snack'] = stock_df['snack'].apply(change_name)
stock_df

In [ ]:
# Another solution, this one with regex
import re

def change_name(snack_name):
    snack_name=snack_name.split("'")
    snack_name=snack_name[1][1:].strip().title() #selecting the second character to the last of the second part of the split, removes space, then makes it in title format
    return re.sub(r'(?<=-)\w+', lambda m: m.group().lower(), snack_name) #also checks for words after _ and converts to lowercase


stock['snack']=stock['snack'].apply(change_name)
stock

### Question 2: Merging the Datasets

Once that both columns look the same, you need to **merge** them as a single data frame so that this can be used in the program.

*Hint:* The merged data frame can have `machine_code` as the index, and all the information from the `stock` data frame next to each product.

In [ ]:
## use this cell to merge stock into vending_machine
df = pd.merge(vending_machine, stock, how="left", on="snack")
df = df.set_index('machine_code')
df

,snack,amount,category,price
machine_code,,,,
A1,NaN,NaN,NaN,NaN
A2,Diet Coke,7.0,drinks,3.75
A3,Coke,2.0,drinks,3.70
A4,Irn-bru,5.0,drinks,3.00
B1,Mars,5.0,chocolate,2.90
B2,Mars,7.0,chocolate,2.90
B3,Milky Way,8.0,chocolate,2.85
B4,NaN,NaN,NaN,NaN
C1,Sea Salt Crisps,2.0,crisps,2.15


### Question 3: Changing values in a data frame

Oops! I forgot to add three Irn-bru cans in B4! Can you do add them to the data frame please?

In [ ]:
## Use this cell to add three units of Irn-bru to slot B4
vending_machine.loc['B4'] = ['Irn-bru',3,'drinks',3.00]
vending_machine

,snack,amount,category,price
machine_code,,,,
A1,NaN,NaN,NaN,NaN
A2,Diet Coke,7.0,drinks,3.75
A3,Coke,2.0,drinks,3.70
A4,Irn-bru,5.0,drinks,3.00
B1,Mars,5.0,chocolate,2.90
B2,Mars,7.0,chocolate,2.90
B3,Milky Way,8.0,chocolate,2.85
B4,Irn-bru,3.0,drinks,3.00
C1,Sea Salt Crisps,2.0,crisps,2.15


In [ ]:
# Saving the final df into a csv file
df.to_csv('df_final.csv')

## Section 2: Vending Machine Simulation Program

Once that you have generated your merged dataset, you will use the following code cell to write a small program with the following *four* options:

1. **Admin login**: This option will simulate how the vending machine operator validates their credentials. When this option is selected, the user should be prompted to input their username and password. The username should be *admin* and the password is be any number smaller than *10*. You should allow the user to try *four* times, if all of them are incorrect, then the program stops and the program cell needs to be run again.

  * Note: Once that an admin has logged in, this option has to be disabled (i.e. the user cannot go back and select this option).

2. **Add products**: This option can only be accessed once option 1 has been completed, otherwise the message *login first!* should be displayed and the program should go back to the main menu. In this option, the admin will be shown the list of snacks and categories in the vending machine. Then, the admin can input a machine code slot to add *one more product* for that certain slot. Afterwards, the program should show the new stock and go back to the main menu.
  
  * Note: Keep in mind that each space in the vending machine can hold a maximum of *eight* products (of the same snack, of course). Also, you cannot add products in the empty slots. Therefore, your program should warn the admin in case that they want to add more products of a specific snack in a slot, or if the admin wants to add products in an empty slot.

3. **Buy snacks**: This option can be accessed by "anyone", so there is no need for a validation. If this option is selected, the user will be shown the list of snacks, amounts and prices. Then, the user will be requested to select one snack based on the `machine_code`. Once the snack is selected, your program must display the price of the selected product. Then, the user will be prompted to pay. To simulate this payment, you will ask the user for an input and write any positive number (if the user inputs something invalid, ask to try again). Then, your program must check this number against the price of the product to be bought. If the input number is larger than the price, then you should return *your change is...* and the subtraction of the payment minus the price of the snack. If the number is equal to the price, then you must return *thanks for paying*. Else, you should output *you need to pay more* and allow the user to write another amount. After the "purchase", you should output to the user the number of products left for that particular snack (therefore, you need to update the data frame!). Notice that if there were zero products left for the snack selected, then you must prompt this to the user in advance before letting them buy, and ask them to select another snack.

4. **Exit**

In [ ]:
## Use this cell to write your code
import numpy as np
ans = True
login = False
while ans:
  print('Welcome to the vending machine. Select an option:')
  print('1. Log in (admin only)')
  print('2. Add products (admin only)')
  print('3. Buy products')
  print('4. Exit')
  option=input('>> ')
  if option == '1':
    if login == False:
      attempts = 0
      ans2 = True
      while ans2:
        if attempts==4:
          print('Number of attempts has been maxed, goodbye!')
          ans = False
          ans2 = False
          ans3 = False
        else:
          print('Please enter your username:')
          username =input('>> ')
          if username == 'admin':
            ans3 = True
            while ans3:
              if attempts==4:
                print('Number of attempts has been maxed, goodbye!')
                ans = False
                ans2 = False
                ans3 = False
              else:
                print('Please enter your password:')
                password =input('>> ')
                try:
                  if float(password) < 10:
                    print('login successful')
                    ans2 = False
                    ans3 = False
                    login = True
                  else:
                    attempts+=1
                    print('invalid password, current attempts are '+str(attempts))        
                except ValueError:
                  attempts+=1
                  print('password should be a number, current attempts are '+str(attempts))  
          else:
            attempts+=1
            print('Invalid username, current attempts are '+str(attempts))
    else:
      print('You already logged in, back to main menu')
      ans2 = False

  elif option == '2':
    if login == True:
          print('This is the current stock of the vending machine:')
          print(df[['snack', 'amount']])
          print('Select a slot to add one more product:')
          ans3 = True
          while ans3:
            toadd = input('>> ')
            try:
              if df.at[toadd,'amount']==8:
                print('this slot has already the max amount permitted, try another one')
              elif pd.isna(df.at[toadd,'snack'])==True:
                print('this slot is empty, try another one')
              else:
                df.at[toadd,'amount']=df.at[toadd,'amount']+1
                print('the product has been added, here is the updated stock:')
                print(df[['snack', 'amount']])
                ans3 = False
            except KeyError:
              print('Invalid slot, try again')
    else:
      print('login first!')

  elif option == '3':
     print('This is the current stock of the vending machine:')
     print(df[['snack', 'amount', 'price']])
     print('Select a product to buy:')
     ans2 = True
     while ans2:
       tobuy = input('>> ')
       try:
         if df.at[tobuy,'amount']==0 or pd.isna(df.at[tobuy,'amount']):
           print('No products left on this slot, try another one')
         else:
           print('Product to buy: '+str(df.at[tobuy,'snack']))
           print('Price: '+str(df.at[tobuy,'price']))
           print('Please enter the amount of money to pay:')
           ans3 = True
           while ans3:
             payment = input('>> ')
             try:
               if float(payment)>0:
                if float(payment)>df.at[tobuy,'price']:
                  print('Your change is '+str(round((float(payment)-df.at[tobuy,'price']),2)))
                  ans3 = False
                  ans2 = False
                  df.at[tobuy,'amount'] = df.at[tobuy,'amount'] - 1
                  print('The number of snacks left in '+str(tobuy)+' is '+str(df.at[tobuy,'amount']))
                elif float(payment)==df.at[tobuy,'price']:
                  print('Thanks for paying')
                  ans3 = False
                  ans2 = False
                  df.at[tobuy,'amount'] = df.at[tobuy,'amount'] - 1
                  print('The number of snacks left in '+str(tobuy)+' is '+str(df.at[tobuy,'amount']))
                else:
                  print('you need to pay more, try again')
               else:
                 print('you inserted "negative" money! try again!')
             except ValueError:
                print('please insert a number')
       except KeyError:
          print('Invalid slot, try again')
  elif option == '4':
    ans = False
  else:
    print('Invalid option. Try again.')

Welcome to the vending machine. Select an option:
1. Log in (admin only)
2. Add products (admin only)
3. Buy products
4. Exit
>> 5
Invalid option. Try again.
Welcome to the vending machine. Select an option:
1. Log in (admin only)
2. Add products (admin only)
3. Buy products
4. Exit
>> 2
login first!
Welcome to the vending machine. Select an option:
1. Log in (admin only)
2. Add products (admin only)
3. Buy products
4. Exit
>> 1
Please enter your username:
>> carlos
Invalid username, current attempts are 1
Please enter your username:
>> admin
Please enter your password:
>> ja
password should be a number, current attempts are 2
Please enter your password:
>> 11
invalid password, current attempts are 3
Please enter your password:
>> 9
login successful
Welcome to the vending machine. Select an option:
1. Log in (admin only)
2. Add products (admin only)
3. Buy products
4. Exit
>> 1
You already logged in, back to main menu
Welcome to the vending machine. Select an option:
1. Log in (admin o

Keep in mind that I will test your program as many times as I want, with the purpose of "crashing it"! So, be sure you are ready to address all possible input fails! Always check for the correct input, and verify that all error messages are shown correctly. Moreover, ensure that I can execute the options of the program in any order I want!